In [1]:
%matplotlib widget
from matplotlib import pyplot as plt
import numpy as np
from scipy import constants

from dbtj import dbtj, gamma

from IPython.display import display
from ipywidgets import interactive, FloatSlider, Checkbox

## Interactive DBTJ model

In [5]:
# define bias range
bias = np.linspace(-2, 2, 256)

# define initial parameters
R1 = FloatSlider(value=1e6, min=1e6, max=10e9, step=1e6, readout_format='.3g', description='R1 ($\Omega$)')
R2 = FloatSlider(value=1e6, min=1e6, max=10e9, step=1e6, readout_format='.3g', description='R2 ($\Omega$)')
C1 = FloatSlider(value=1., min=0.01, max=5., step=0.01, readout_format='.3g', description='C1 (aF)')
C2 = FloatSlider(value=1., min=0.01, max=5., step=0.01, readout_format='.3g', description='C2 (aF)')
Q0 = FloatSlider(value=0., min=-0.5, max=0.5, step=0.01, description='Q0 (#e)')
T = FloatSlider(value=77., min=3., max=293., step=1., description='T (K)')

# do initial plot
fig, ax = plt.subplots()
line, = ax.plot(bias, bias)
ax.set_xlim(bias.min(), bias.max())
ax.set_xlabel('Bias/ V')
ax.set_ylabel('Current/ nA')

axdiff = ax.twinx()
line2, = axdiff.plot(bias, bias, color='r')
axdiff.set_ylabel('dI/dV')

ax.legend((line, line2), ('Current', 'dI/dV'), loc='lower right')

# pad fraction
pad = plt.rcParams['axes.ymargin']

# interactive function just updates line ydata
def update(R1, R2, C1, C2, Q0, T, diff, ylim):
    current = dbtj(bias, R1, R2, C1*constants.atto, C2*constants.atto, Q0, T)
    # convert to nA
    current /= constants.nano
    # update line
    line.set_ydata(current)
    # create mask to get valid points and exclude infs and NaNs from ylim update
    mask = np.isfinite(current)
    # if dIdV requested then plot
    if diff:
        line2.set_ydata(np.gradient(current, bias))
    # otherwise set invisible
    else:
        line2.set_visible(False)
    # update axes limits if required
    if not ylim:
        ax.set_ylim(current[mask].min()*(1+pad), current[mask].max()*(1+pad))
        axdiff.set_ylim(line2.get_ydata()[mask].min(), line2.get_ydata()[mask].max()*(1+pad))
        
fig.tight_layout()
# create interactive figure
display(interactive(update, R1=R1, R2=R2, C1=C1, C2=C2, Q0=Q0, T=T,
                    diff=Checkbox(value=True, description='dI/dV'),
                    ylim=Checkbox(value=False, description='Fix ylim')))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1000000.0, description='R1 ($\\Omega$)', max=10000000000.0, min=100000…